In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
from sklearn.metrics import classification_report
import numpy as np

Descrição da rede neural para o MNIST
<blockquote>We adopt a simple feed-forward architecture with three hidden layers. <b>The first convolutional layer has 32 channels with 5 × 5 kernels</b> followed by <b>3 × 3 max pooling and a stride of 2</b>. <b>The second layer is also convolutional, again with 32 channels with 3 × 3 kernels, also followed by 3 × 3 max pooling and a stride of 2</b>. This leads to a 1,568 dimensional representation <b>which feeds to a 100-dimensional tanh hidden layer with fully-connected weights, which then in a fully-connected manner feeds to the ten outputs</b>. We have softmax outputs and minimize <b>cross-entropy using Adam update rule with a learning rate 10−3</b>. The updates are performed after each <b>mini-batch of size 64</b>. The network for MNIST is trained for <b>25 epochs</b>. On MNIST, the model is able to achieve 99.25% accuracy on the test set.</blockquote>

In [3]:
# MNIST carregado
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [4]:
# Pré-processamento -- normalizando entre 0 e 1
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test  = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [5]:
model = models.Sequential([
    layers.Input(shape=(28, 28, 1), name="input_layer"),
    # Camada convolucional 1
    layers.Conv2D(32, (5, 5), padding='same', activation='relu', name="conv1"),
    layers.MaxPooling2D(pool_size=(3, 3), strides=2, name="pool1"),

    # Camada convolucional 2
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', name="conv2"),
    layers.MaxPooling2D(pool_size=(3, 3), strides=2, name="pool2"),

    # Flatten
    layers.Flatten(name="flatten"),

    # Camada fully connected com tanh
    layers.Dense(100, activation='tanh', name="dense_tanh"),

    # Camada de saída com softmax
    layers.Dense(10, activation='softmax', name="output_layer")
], name="mnist_model")

2025-06-08 17:34:56.166495: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

model.summary()

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 28, 28, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_tanh (Dense)              │ (None, 100)            │       115,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,390 (493.71 KB)

 Trainable params: 126,390 (493.71 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=25,
    batch_size=64,
    callbacks=[early_stopping]
)

Epoch 1/25


2025-06-08 17:34:59.839063: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8865 - loss: 0.3670 - val_accuracy: 0.9843 - val_loss: 0.0489
Epoch 2/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9863 - loss: 0.0451 - val_accuracy: 0.9888 - val_loss: 0.0350
Epoch 3/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9915 - loss: 0.0289 - val_accuracy: 0.9899 - val_loss: 0.0291
Epoch 4/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9930 - loss: 0.0235 - val_accuracy: 0.9902 - val_loss: 0.0299
Epoch 5/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9949 - loss: 0.0170 - val_accuracy: 0.9907 - val_loss: 0.0301
Epoch 6/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9956 - loss: 0.0153 - val_accuracy: 0.9913 - val_loss: 0.0273
Epoch 7/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9965 - loss: 0.0113 - val_accuracy: 0.9926 - val_loss: 0.0256
Epoch 8/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9967 - loss: 0.0106 - val_accurac

In [8]:
# Salvar o modelo
model.save("modelos/mnist_model.keras")

In [9]:
# relatorio de treinamento
y_pred = np.argmax(model.predict(x_test), axis=1)
print(classification_report(y_test, y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       0.99      1.00      0.99      1032
           3       0.99      1.00      1.00      1010
           4       0.99      1.00      0.99       982
           5       0.99      1.00      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [ ]:
# acurácia alcançada
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Acurácia no conjunto de teste: {test_accuracy:.5f}") # melhor do que a do artigo, 99,25%

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9923 - loss: 0.0288
Acurácia no conjunto de teste: 0.99360
